In [1]:
# This code is for checking 
# Preprocessing method for 법정동

In [2]:
from glob import glob

import pandas as pd
import geopandas as gpd

In [3]:
bdong_shp_paths = glob("./data/raw_data/법정동_좌표/*/*.shp")
bdong_shp_paths

['./data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_충남/LSMD_ADM_SECT_UMD_44_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_울산/LSMD_ADM_SECT_UMD_31_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_경남/LSMD_ADM_SECT_UMD_48_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_대구/LSMD_ADM_SECT_UMD_27_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_부산/LSMD_ADM_SECT_UMD_26_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_서울/LSMD_ADM_SECT_UMD_11_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_대전/LSMD_ADM_SECT_UMD_30_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_전남/LSMD_ADM_SECT_UMD_46_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_전북특별자치도/LSMD_ADM_SECT_UMD_52_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_강원특별자치도/LSMD_ADM_SECT_UMD_51_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_경북/LSMD_ADM_SECT_UMD_47_202403.shp',
 './data/raw_data/법정동_좌표/LSMD_ADM_SECT_UMD_충북/LSMD_AD

In [4]:
def read_gdf(file_path, encoding):
    gdf = gpd.read_file(file_path, encoding=encoding)
    return gdf


def gdf2df(gdf):
    return pd.DataFrame(gdf)


read_gdf_with_cp949 = lambda file_path: read_gdf(file_path, "cp949")
read_shp = lambda file_path: gdf2df(read_gdf_with_cp949(file_path))

In [5]:
bdong_shp_df = pd.concat([read_shp(shp_path) for shp_path in bdong_shp_paths])
bdong_shp_df.head()

,EMD_CD,COL_ADM_SE,EMD_NM,SGG_OID,geometry
0,44131101,44130,대흥동,8151,"POLYGON ((213016.846 467225.891, 213018.764 46..."
1,44131102,44130,성황동,8185,"POLYGON ((213172.114 468227.275, 213176.304 46..."
2,44131103,44130,문화동,8183,"POLYGON ((213242.494 467608.139, 213242.699 46..."
3,44131104,44130,사직동,108,"POLYGON ((213355.277 466524.999, 213333.886 46..."
4,44131105,44130,영성동,8010,"POLYGON ((213333.372 466707.548, 213333.413 46..."


In [6]:
def read_csv(file_path, params):
    return pd.read_csv(file_path, **params)

read_csv_params = {
    "dtype": {"시도코드": str, "시군구코드": str, "법정동코드": str},
}

read_csv_with_params = lambda file_path: read_csv(file_path, read_csv_params)

In [7]:
file_path = "./data/preproc_data/bdong_raw.csv"
bdong_base = read_csv_with_params(file_path)
bdong_base.head()

,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,NaN,1100000000,NaN,NaN,19880423,NaN
1,11,서울특별시,11110,종로구,1111000000,NaN,NaN,19880423,NaN
2,11,서울특별시,11110,종로구,1111010100,청운동,NaN,19880423,NaN
3,11,서울특별시,11110,종로구,1111010200,신교동,NaN,19880423,NaN
4,11,서울특별시,11110,종로구,1111010300,궁정동,NaN,19880423,NaN


In [8]:
bdong_base["rough_name"] = bdong_base["시도명"] + "_" + bdong_base["시군구명"].fillna("")
rough_code_name_dict = bdong_base.set_index("시군구코드")['rough_name'].to_dict()

In [9]:
bdong_shp_df["rough_name"] = bdong_shp_df["COL_ADM_SE"].astype(str).map(rough_code_name_dict)

In [10]:
preproc_bdong_shp_df = bdong_shp_df.loc[:,['EMD_NM','EMD_CD','rough_name','geometry']]
preproc_bdong_shp_df.columns = ["detail_name", "detail_code", "rough_name", "geometry"]